<a href="https://www.kaggle.com/code/kuncoroaji/segmenting-and-clustering-in-east-jakarta?scriptVersionId=115852892" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in East Jakarta</font></h1>

## 1. INTRODUCTION

### 1.1. Background

In the past decade, the lifestyle of urban people has changed with the trends and habits of drinking coffee. Coffee, which was ancient, is identical to drinks commonly used by older men, now women and men of all ages are accustomed to drinking coffee. And not just enjoying coffee, but many people are looking for a place to drink coffee. The coffee shop has finally become a cool hangout with an internet connection while enjoying a variety of steeping coffee beans. 
This coffee drinking trend will become a big business opportunity. The business world is starting to work on places that serve specialty coffee. Although the Indonesian people are not addicted to coffee, which means they have to drink every day, like in Melbourne. And the Coffe shop industry is still relatively new, but in big cities like Jakarta, Coffe shop has the opportunity to get a gross profit of Rp 100 million to Rp 1 billion. However, getting into the business world is not as easy as one might imagine.
If you already have the capital to open a coffee shop, then you must have the courage, start designing strategies and seeing the market. If you have long been in love with coffee and a hobby of drinking coffee, it means you can start a business with the right passion. Therefore I try to practice my learning at Coursera to answer relevant questions, namely designing strategies to determine which areas are suitable for opening coffee shops.

### 1.2. Problem
Finding data about the area and postcode in South Jakarta is a challenge that must be resolved. The
price of renting a place to determine the exact location of a coffee shop is also one of the problems that
must be resolved.

### 1.3. Interest
I believe this is a relevant challenge with a valid question for anyone who wants to open a coffee shop and determine the right location. The same methodology can be applied according to demands as applicable. This case also applies to anyone interested in exploring starting or finding new business in any city. Finally, this can also serve as a good practical exercise for developing Data Science skills.

## 2. Data Acquisition and Cleaning

### 2.1. Data Acquisition
1. The data acquired for this project is a combination of data from two sources. The first data source of data is scraped from a wikipedia page that contains the list of Neighboorhod East Jakarta ---> https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Administrasi_Jakarta_Selatan. This page contains additional information about the boroughs, the following are the columns:
    * Kelurahan : Name of the urban village
    * Kecamatan : Name of the sub-district
    * Kota / Provinsi : Name of the province



2. The Second data source is the list of Logitude & Latitude from website longlat.net, and list of postcode from Wikipedia in East Jakartan, the following are columns:

    * Kelurahan : Name of the urban village.
    * PostCode : Number of the posccode area.
    * Latitude : Latitude of the urban village.
    * Longitude : Longitude of the urban village.

#### Import necessary libraries

In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... - done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=6578dac862d6f5c505d5f110eb3f25db01987e0aeaa61da13126423271b5ae49
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

import wikipedia as wp

import warnings

print('Libraries imported.')

Libraries imported.


### 2.2. Download and Explore Dataset 

In [3]:
import requests
website_url = requests.get('https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Administrasi_Jakarta_Selatan').text

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="id">
 <head>
  <meta charset="utf-8"/>
  <title>
   Daftar kecamatan dan kelurahan di Kota Administrasi Jakarta Selatan - Wikipedia bahasa Indonesia, ensiklopedia bebas
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januari","Februari","Maret","April","Mei","Juni","Juli","Agustus","September","Oktober","November","Desember"],"wgRequestId":"07f84aed-119d-411c-9202-3b7065aad15a","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Daftar_kecamatan_dan_kelurahan_di_Kota_Administrasi_Jakarta_Selatan","wgTitle":"Daftar kecamatan dan kelurahan di Kota Administrasi Jakarta Selatan","wgCurRevisionId":19428605,"wgRevisionId":19428605,"wgArticleId":598036,"wgIsArticle":true,"wgIsRedirect":false,"wgAct

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr style="background-color: #ccc;" valign="top">
<th>Kode <br/>Kemendagri</th>
<th>Kecamatan</th>
<th>Jumlah <br/>Kelurahan</th>
<th>Daftar <br/>Kelurahan
</th></tr>
<tr valign="top">
<td>31.74.06
</td>
<td><a href="/wiki/Cilandak,_Jakarta_Selatan" title="Cilandak, Jakarta Selatan">Cilandak</a></td>
<td align="center">5</td>
<td><div class="hlist">
<ul><li><a href="/wiki/Cilandak_Barat,_Cilandak,_Jakarta_Selatan" title="Cilandak Barat, Cilandak, Jakarta Selatan">Cilandak Barat</a></li>
<li><a href="/wiki/Cipete_Selatan,_Cilandak,_Jakarta_Selatan" title="Cipete Selatan, Cilandak, Jakarta Selatan">Cipete Selatan</a></li>
<li><a href="/wiki/Gandaria_Selatan,_Cilandak,_Jakarta_Selatan" title="Gandaria Selatan, Cilandak, Jakarta Selatan">Gandaria Selatan</a></li>
<li><a href="/wiki/Lebak_Bulus,_Cilandak,_Jakarta_Selatan" title="Lebak Bulus, Cilandak, Jakarta Selatan">Lebak Bulus</a></li>
<li><a href="/wiki/Pondok_Labu,_Cilandak,_Jakarta_Selatan" ti

In [6]:
links = My_table.findAll('a')
links

[<a href="/wiki/Cilandak,_Jakarta_Selatan" title="Cilandak, Jakarta Selatan">Cilandak</a>,
 <a href="/wiki/Cilandak_Barat,_Cilandak,_Jakarta_Selatan" title="Cilandak Barat, Cilandak, Jakarta Selatan">Cilandak Barat</a>,
 <a href="/wiki/Cipete_Selatan,_Cilandak,_Jakarta_Selatan" title="Cipete Selatan, Cilandak, Jakarta Selatan">Cipete Selatan</a>,
 <a href="/wiki/Gandaria_Selatan,_Cilandak,_Jakarta_Selatan" title="Gandaria Selatan, Cilandak, Jakarta Selatan">Gandaria Selatan</a>,
 <a href="/wiki/Lebak_Bulus,_Cilandak,_Jakarta_Selatan" title="Lebak Bulus, Cilandak, Jakarta Selatan">Lebak Bulus</a>,
 <a href="/wiki/Pondok_Labu,_Cilandak,_Jakarta_Selatan" title="Pondok Labu, Cilandak, Jakarta Selatan">Pondok Labu</a>,
 <a href="/wiki/Jagakarsa,_Jakarta_Selatan" title="Jagakarsa, Jakarta Selatan">Jagakarsa</a>,
 <a href="/wiki/Ciganjur,_Jagakarsa,_Jakarta_Selatan" title="Ciganjur, Jagakarsa, Jakarta Selatan">Ciganjur</a>,
 <a href="/wiki/Cipedak,_Jagakarsa,_Jakarta_Selatan" title="Cipedak, 

In [7]:
Provinsi = []
for link in links:
    Provinsi.append(link.get('title'))
    
print(Provinsi)

['Cilandak, Jakarta Selatan', 'Cilandak Barat, Cilandak, Jakarta Selatan', 'Cipete Selatan, Cilandak, Jakarta Selatan', 'Gandaria Selatan, Cilandak, Jakarta Selatan', 'Lebak Bulus, Cilandak, Jakarta Selatan', 'Pondok Labu, Cilandak, Jakarta Selatan', 'Jagakarsa, Jakarta Selatan', 'Ciganjur, Jagakarsa, Jakarta Selatan', 'Cipedak, Jagakarsa, Jakarta Selatan', 'Jagakarsa, Jagakarsa, Jakarta Selatan', 'Lenteng Agung, Jagakarsa, Jakarta Selatan', 'Srengseng Sawah, Jagakarsa, Jakarta Selatan', 'Tanjung Barat, Jagakarsa, Jakarta Selatan', 'Kebayoran Baru, Jakarta Selatan', 'Cipete Utara, Kebayoran Baru, Jakarta Selatan', 'Gandaria Utara, Kebayoran Baru, Jakarta Selatan', 'Gunung, Kebayoran Baru, Jakarta Selatan', 'Kramat Pela, Kebayoran Baru, Jakarta Selatan', 'Melawai, Kebayoran Baru, Jakarta Selatan', 'Petogogan, Kebayoran Baru, Jakarta Selatan', 'Pulo, Kebayoran Baru, Jakarta Selatan', 'Rawa Barat, Kebayoran Baru, Jakarta Selatan', 'Selong, Kebayoran Baru, Jakarta Selatan', 'Senayan, Kebay

In [8]:
df = pd.DataFrame()
df['Provinsi'] = Provinsi
df = df.Provinsi.str.split(',', expand=True).rename(columns={0: "Kelurahan", 1: "Kecamatan", 2: "Kota"}, errors="raise").dropna()
df.head()

,Kelurahan,Kecamatan,Kota
1,Cilandak Barat,Cilandak,Jakarta Selatan
2,Cipete Selatan,Cilandak,Jakarta Selatan
3,Gandaria Selatan,Cilandak,Jakarta Selatan
4,Lebak Bulus,Cilandak,Jakarta Selatan
5,Pondok Labu,Cilandak,Jakarta Selatan


In [9]:
East_jakarta_geo = pd.read_csv('../input/east-jakarta-geo/East_Jakarta.csv', sep = ';')
East_jakarta_geo = East_jakarta_geo.filter(["PostCode", "Kelurahan", "Latitude","Longitude"])
East_jakarta_geo.head()

,PostCode,Kelurahan,Latitude,Longitude
0,12430,Cilandak Barat,-6.288289,106.796765
1,12410,Cipete Selatan,-6.271827,106.804876
2,12420,Gandaria Selatan,-6.272557,106.794845
3,12440,Lebak Bulus,-6.301837,106.779642
4,12450,Pondok Labu,-6.308832,106.797495


In [10]:
East_jakarta_merged = pd.merge(df, East_jakarta_geo, on='Kelurahan')
East_jakarta_merged.head()

,Kelurahan,Kecamatan,Kota,PostCode,Latitude,Longitude
0,Cilandak Barat,Cilandak,Jakarta Selatan,12430,-6.288289,106.796765
1,Cipete Selatan,Cilandak,Jakarta Selatan,12410,-6.271827,106.804876
2,Gandaria Selatan,Cilandak,Jakarta Selatan,12420,-6.272557,106.794845
3,Lebak Bulus,Cilandak,Jakarta Selatan,12440,-6.301837,106.779642
4,Pondok Labu,Cilandak,Jakarta Selatan,12450,-6.308832,106.797495


In [11]:
East_jakarta_data=East_jakarta_merged[['PostCode','Kecamatan','Kelurahan','Latitude','Longitude']]
East_jakarta_data

,PostCode,Kecamatan,Kelurahan,Latitude,Longitude
0,12430,Cilandak,Cilandak Barat,-6.288289,106.796765
1,12410,Cilandak,Cipete Selatan,-6.271827,106.804876
2,12420,Cilandak,Gandaria Selatan,-6.272557,106.794845
3,12440,Cilandak,Lebak Bulus,-6.301837,106.779642
4,12450,Cilandak,Pondok Labu,-6.308832,106.797495
5,12620,Jagakarsa,Ciganjur,-6.335664,106.807269
6,12630,Jagakarsa,Cipedak,-6.351934,106.801658
7,12620,Jagakarsa,Jagakarsa,-6.324892,106.819768
8,12610,Jagakarsa,Lenteng Agung,-6.323122,106.836247
9,12640,Jagakarsa,Srengseng Sawah,-6.344935,106.826291


### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'ZCDBYWTWVFRJTKKW3NYERHNPNA3TA2J5A0L4TJGWDMYSD5ER' # your Foursquare ID
CLIENT_SECRET = 'Q5M4N2IPD5Z3RPUDKU1CQWNVU1B5YV01CNOXNNXPPTZO5YO5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZCDBYWTWVFRJTKKW3NYERHNPNA3TA2J5A0L4TJGWDMYSD5ER
CLIENT_SECRET:Q5M4N2IPD5Z3RPUDKU1CQWNVU1B5YV01CNOXNNXPPTZO5YO5


#### Let's explore the first neighborhood in our dataframe.

In [13]:
East_jakarta_data.loc[0, 'Kelurahan']

'Cilandak Barat'

Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = East_jakarta_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = East_jakarta_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = East_jakarta_data.loc[0, 'Kelurahan'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cilandak Barat are -6.288289, 106.796765.


### Now, let's get the top 100 venues in  radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZCDBYWTWVFRJTKKW3NYERHNPNA3TA2J5A0L4TJGWDMYSD5ER&client_secret=Q5M4N2IPD5Z3RPUDKU1CQWNVU1B5YV01CNOXNNXPPTZO5YO5&v=20180605&ll=-6.288289,106.796765&radius=500&limit=100'

Send the GET request and examine the resutls

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '63bb5e0277ad8d1acc04e37f'},
 'response': {'queryRefinements': {'target': {'type': 'path',
    'url': '/venue/explore',
    'params': {'ll': '-6.288289,106.796765', 'radius': '500'}},
   'refinements': [{'query': 'Food'},
    {'query': 'Nightlife'},
    {'query': 'Coffee'},
    {'query': 'Shops'},
    {'query': 'Arts'},
    {'query': 'Outdoors'}]},
  'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cilandak',
  'headerFullLocation': 'Cilandak, Jakarta',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 36,
  'suggestedBounds': {'ne': {'lat': -6.283788995499996,
    'lng': 106.80128379039412},
   'sw': {'lat': -6.292789004500004, 'lng': 106.79224620960586}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reason

**get_category_type** function from the Foursquare lab.

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Total Buah Segar,Farmers Market,-6.287626,106.795414
1,Roti Bakar Wiwied,Sandwich Place,-6.289398,106.795378
2,Maxima Fitness,Gym,-6.287644,106.795485
3,Sate Padang Salero Ajo,Food Truck,-6.287513,106.795367
4,Mars Kitchen,Café,-6.287163,106.795424


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

36 venues were returned by Foursquare.


## 3. Explore Neighborhoods in East Jakarta

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Kelurahan', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [21]:
East_jakarta_venues = getNearbyVenues(names=East_jakarta_data['Kelurahan'],
                                   latitudes=East_jakarta_data['Latitude'],
                                   longitudes=East_jakarta_data['Longitude']
                                  )

Cilandak Barat
Cipete Selatan
Gandaria Selatan
Lebak Bulus
Pondok Labu
Ciganjur
Cipedak
Jagakarsa
Lenteng Agung
Srengseng Sawah
Tanjung Barat
Cipete Utara
Gandaria Utara
Gunung
Kramat Pela
Melawai
Petogogan
Pulo
Rawa Barat
Selong
Senayan
Cipulir
Grogol Selatan
Grogol Utara
Kebayoran Lama Selatan
Kebayoran Lama Utara
Pondok Pinang
Bangka
Kuningan Barat
Mampang Prapatan
Pela Mampang
Tegal Parang
Cikoko
Duren Tiga
Kalibata
Pancoran
Pengadegan
Rawajati
Cilandak Timur
Jati Padang
Kebagusan
Pasar Minggu
Pejaten Barat
Pejaten Timur
Ragunan
Bintaro
Pesanggrahan
Petukangan Selatan
Petukangan Utara
Guntur
Karet Kuningan
Karet Semanggi
Karet
Kuningan Timur
Menteng Atas
Pasar Manggis
Setiabudi
Bukit Duri
Kebon Baru
Manggarai Selatan
Manggarai
Menteng Dalam
Tebet Barat
Tebet Timur


In [22]:
print(East_jakarta_venues.shape)
East_jakarta_venues.head()

(1077, 7)


,Kelurahan,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cilandak Barat,-6.288289,106.796765,Total Buah Segar,-6.287626,106.795414,Farmers Market
1,Cilandak Barat,-6.288289,106.796765,Roti Bakar Wiwied,-6.289398,106.795378,Sandwich Place
2,Cilandak Barat,-6.288289,106.796765,Maxima Fitness,-6.287644,106.795485,Gym
3,Cilandak Barat,-6.288289,106.796765,Sate Padang Salero Ajo,-6.287513,106.795367,Food Truck
4,Cilandak Barat,-6.288289,106.796765,Mars Kitchen,-6.287163,106.795424,Café


Let's check how many venues were returned for each neighborhood

In [23]:
East_jakarta_venues.groupby('Kelurahan').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Kelurahan,,,,,,
Bangka,10,10,10,10,10,10
Bintaro,14,14,14,14,14,14
Bukit Duri,2,2,2,2,2,2
Ciganjur,3,3,3,3,3,3
Cikoko,13,13,13,13,13,13
Cilandak Barat,36,36,36,36,36,36
Cilandak Timur,8,8,8,8,8,8
Cipedak,6,6,6,6,6,6
Cipete Selatan,8,8,8,8,8,8


#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(East_jakarta_venues['Venue Category'].unique())))

There are 179 uniques categories.


## 4. Analyze Each Neighborhood

In [25]:
# one hot encoding
East_jakarta_onehot = pd.get_dummies(East_jakarta_venues[['Venue Category']], prefix="", prefix_sep="")
#Mampang_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
East_jakarta_onehot.insert(loc=0, column='Kelurahan', value=East_jakarta_venues['Kelurahan'] )
East_jakarta_onehot.shape

(1077, 180)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
East_jakarta_grouped = East_jakarta_onehot.groupby('Kelurahan').mean().reset_index()
East_jakarta_grouped.head()

,Kelurahan,Acehnese Restaurant,Airport,American Restaurant,Apres Ski Bar,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Balinese Restaurant,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Campground,Car Wash,Chinese Restaurant,Clothing Store,Coffee Shop,College Residence Hall,Comfort Food Restaurant,Concert Hall,Convenience Store,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,High School,History Museum,Hobby Shop,Hospital,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Light Rail Station,Lighthouse,Lounge,Manadonese Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Student Center,Sundanese Restaurant,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tea Room,Tech Startup,Thai Restaurant,Theme Restaurant,Train Station,Travel Agency,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Bangka,0.0,0.0,0.000000,0.0,0.000000,0.1,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.300000,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bintaro,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

#### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in East_jakarta_grouped['Kelurahan']:
    print("----"+hood+"----")
    temp = East_jakarta_grouped[East_jakarta_grouped['Kelurahan'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bangka----
                       venue  freq
0                Coffee Shop   0.3
1  Middle Eastern Restaurant   0.1
2                 Restaurant   0.1
3                Art Gallery   0.1
4        Arts & Crafts Store   0.1


----Bintaro----
                 venue  freq
0           Food Truck  0.14
1    Convenience Store  0.14
2           Hobby Shop  0.07
3               Bakery  0.07
4  American Restaurant  0.07


----Bukit Duri----
                   venue  freq
0  Indonesian Restaurant   0.5
1       Asian Restaurant   0.5
2    Acehnese Restaurant   0.0
3  Padangnese Restaurant   0.0
4            Music Store   0.0


----Ciganjur----
                   venue  freq
0             Steakhouse  0.33
1                 Arcade  0.33
2      Convenience Store  0.33
3  Padangnese Restaurant  0.00
4            Music Store  0.00


----Cikoko----
               venue  freq
0        Coffee Shop  0.15
1  Convenience Store  0.15
2      Train Station  0.08
3        Gas Station  0.08
4        Supermarke

#### Let's put that into a *pandas* dataframe

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Kelurahan']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Kelurahan'] = East_jakarta_grouped['Kelurahan']

for ind in np.arange(East_jakarta_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(East_jakarta_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangka,Coffee Shop,Middle Eastern Restaurant,Restaurant,Art Gallery,Arts & Crafts Store,Bistro,Convenience Store,Pet Service,Acehnese Restaurant,New American Restaurant
1,Bintaro,Food Truck,Convenience Store,Hobby Shop,Bakery,American Restaurant,Restaurant,Burger Joint,Pizza Place,Asian Restaurant,Indonesian Restaurant
2,Bukit Duri,Indonesian Restaurant,Asian Restaurant,Acehnese Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
3,Ciganjur,Steakhouse,Arcade,Convenience Store,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
4,Cikoko,Coffee Shop,Convenience Store,Train Station,Gas Station,Supermarket,Boat or Ferry,Asian Restaurant,Gym / Fitness Center,Hardware Store,Chinese Restaurant


## 5. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [30]:

# set number of clusters
kclusters = 5

East_jakarta_grouped_clustering = East_jakarta_grouped.drop('Kelurahan', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(East_jakarta_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


array([0, 1, 1, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

East_jakarta_merged = East_jakarta_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
East_jakarta_merged = East_jakarta_merged.join(neighborhoods_venues_sorted.set_index('Kelurahan'), on='Kelurahan')

East_jakarta_merged.head()

,PostCode,Kecamatan,Kelurahan,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,12430,Cilandak,Cilandak Barat,-6.288289,106.796765,1,Coffee Shop,Bookstore,Indonesian Restaurant,Fast Food Restaurant,Pizza Place,Café,Hardware Store,Student Center,Gym,Spa
1,12410,Cilandak,Cipete Selatan,-6.271827,106.804876,1,Food Court,Music Venue,Indonesian Restaurant,Spa,Café,Auto Workshop,Coffee Shop,Neighborhood,New American Restaurant,Nightclub
2,12420,Cilandak,Gandaria Selatan,-6.272557,106.794845,1,Indonesian Restaurant,Café,Karaoke Bar,Fried Chicken Joint,Motorcycle Shop,Food Truck,Clothing Store,Acehnese Restaurant,Padangnese Restaurant,New American Restaurant
3,12440,Cilandak,Lebak Bulus,-6.301837,106.779642,1,Convenience Store,Asian Restaurant,Café,Sushi Restaurant,Indonesian Restaurant,Soup Place,Shabu-Shabu Restaurant,Restaurant,Bubble Tea Shop,Food Truck
4,12450,Cilandak,Pondok Labu,-6.308832,106.797495,4,Grocery Store,Javanese Restaurant,Fast Food Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House


In [32]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bangka,Coffee Shop,Middle Eastern Restaurant,Restaurant,Art Gallery,Arts & Crafts Store,Bistro,Convenience Store,Pet Service,Acehnese Restaurant,New American Restaurant
1,1,Bintaro,Food Truck,Convenience Store,Hobby Shop,Bakery,American Restaurant,Restaurant,Burger Joint,Pizza Place,Asian Restaurant,Indonesian Restaurant
2,1,Bukit Duri,Indonesian Restaurant,Asian Restaurant,Acehnese Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
3,0,Ciganjur,Steakhouse,Arcade,Convenience Store,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
4,0,Cikoko,Coffee Shop,Convenience Store,Train Station,Gas Station,Supermarket,Boat or Ferry,Asian Restaurant,Gym / Fitness Center,Hardware Store,Chinese Restaurant


Finally, let's visualize the resulting clusters

In [33]:
latitude = -6.260838
longitude = 106.820788
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(East_jakarta_merged['Latitude'], East_jakarta_merged['Longitude'], East_jakarta_merged['Kelurahan'], East_jakarta_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters
Elimination of Coffee Shop and Café place categories because of avoiding competition with other coffee shops.

In [34]:
One = neighborhoods_venues_sorted[neighborhoods_venues_sorted["1st Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
two = One[One["2nd Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
three =two[two["3rd Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
four = three[three["4th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
five = four[four["5th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
six = five[five["6th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
seven = six[six["7th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
eight = seven[seven["8th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
nine = eight[eight["9th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]
Location_Recomendation = nine[nine["10th Most Common Venue"].apply(lambda x:x not in ['Coffee Shop','Café'])]

### Cluster 1

In [35]:
Location_Recomendation.loc[Location_Recomendation['Cluster Labels'] == 0]

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,0,Ciganjur,Steakhouse,Arcade,Convenience Store,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
18,0,Jagakarsa,Acehnese Restaurant,Garden,Convenience Store,Pharmacy,Padangnese Restaurant,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
45,0,Pengadegan,BBQ Joint,Boat or Ferry,Noodle House,Convenience Store,Food Truck,Car Wash,Paper / Office Supplies Store,Neighborhood,New American Restaurant,Nightclub


### Cluster 2

In [36]:
Location_Recomendation.loc[Location_Recomendation['Cluster Labels'] == 1]

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,1,Bintaro,Food Truck,Convenience Store,Hobby Shop,Bakery,American Restaurant,Restaurant,Burger Joint,Pizza Place,Asian Restaurant,Indonesian Restaurant
2,1,Bukit Duri,Indonesian Restaurant,Asian Restaurant,Acehnese Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
7,1,Cipedak,Juice Bar,Department Store,Convenience Store,Motorcycle Shop,Farmers Market,Grocery Store,Padangnese Restaurant,Neighborhood,New American Restaurant,Nightclub
10,1,Cipulir,Asian Restaurant,Hobby Shop,Sporting Goods Shop,Department Store,Restaurant,Hotel,Other Great Outdoors,Padangnese Restaurant,Music Venue,Neighborhood
14,1,Grogol Selatan,Indonesian Restaurant,Apres Ski Bar,Soup Place,Acehnese Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub
19,1,Jati Padang,Japanese Restaurant,Asian Restaurant,Convenience Store,Food Truck,Breakfast Spot,High School,Fast Food Restaurant,Music Store,Music Venue,Neighborhood
26,1,Kebayoran Lama Utara,Donut Shop,Art Gallery,Tailor Shop,Golf Course,Acehnese Restaurant,Office,Music School,Music Store,Music Venue,Neighborhood
27,1,Kebon Baru,Indonesian Restaurant,Indonesian Meatball Place,Housing Development,Grocery Store,Noodle House,Acehnese Restaurant,Other Great Outdoors,Music Store,Music Venue,Neighborhood
34,1,Manggarai,Arcade,Japanese Restaurant,Fast Food Restaurant,Moving Target,Indonesian Restaurant,Paper / Office Supplies Store,Music Venue,Neighborhood,New American Restaurant,Nightclub
35,1,Manggarai Selatan,Clothing Store,Indonesian Restaurant,Arcade,Paper / Office Supplies Store,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House


### Cluster 3

In [37]:
Location_Recomendation.loc[Location_Recomendation['Cluster Labels'] == 2]

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,2,Srengseng Sawah,College Residence Hall,Acehnese Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office


### Cluster 4

In [38]:
Location_Recomendation.loc[Location_Recomendation['Cluster Labels'] == 3]

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,3,Grogol Utara,Noodle House,Acehnese Restaurant,Pub,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office


### Cluster 5

In [39]:
Location_Recomendation.loc[Location_Recomendation['Cluster Labels'] == 4]

,Cluster Labels,Kelurahan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,4,Pesanggrahan,Grocery Store,Food Truck,Acehnese Restaurant,Other Great Outdoors,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House
50,4,Pondok Labu,Grocery Store,Javanese Restaurant,Fast Food Restaurant,Padangnese Restaurant,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House


## 7. Results and Discussion
The purpose of this project is to help people or coffee shop owners who want to open a new shop in an area by comparing the number of coffee shops in the area. The right area to open coffee shops for the first time is in clusters 3, 4 or 5 because the venue categories such as Food Trucks, Food Stands and Food Court indirectly there is a possibility of coffee menus in the venue and still a little even mostly above the 5th Most Common Venue. But if you want to open branches or add franchises to clusters 1 and 2 can be a consideration.

## 8. Conclusion 
This project helps one get a better understanding of the environment in relation to the most suitable place to open coffee shops. The future of this project includes considering other factors such as the cost of renting a place, the price of land to open a new coffee shop or even the work and salaries of each person in the area to be able to more accurately determine the price of coffee to be sold.